In [60]:
import pandas as pd 

ibge = pd.read_csv("/home/lisamenezes/Searches/pipeline-dados-PDM/data/silver/ibge_goiania.csv")

tse = pd.read_csv("/home/lisamenezes/pipeline-dados-PDM/data/data/silver/tse_bairros_2016.csv")
# ibge = ibge[:300]
# tse = tse[:300]

In [61]:
ibge.head()

,Unnamed: 0,Nome_do_subdistrito,Nome_do_bairro,Situacao_setor,V001,V002,V005,V007,V009,V011
0,3239,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,276.0,642.0,"2432,47","2543,03","1588,1","2084,38"
1,3240,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,225.0,491.0,"2198,14","2389,29","1512,69","2008,52"
2,3241,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,250.0,584.0,"1787,44","1885,48","1233,05","1513,68"
3,3242,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,217.0,363.0,"2209,32","2282,96","1730,26","2051,34"
4,3243,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,214.0,571.0,"3044,29","3257,39","1871,55","2607,5"


In [62]:
tse.head()

,Unnamed: 0,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO
0,0,2016,GOIANIA,133,146,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,1,COLEGIO ESTADUAL MAJOR OSCAR ALVEOLOS,"RUA 2011, UNIDADE 201, S/N, PARQUE ATHENEU","['COLEGIO DINAMICO', 'COLEGIO MARISTA', 'CENTR...","['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ..."
1,1,2016,GOIANIA,2,42,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,3,ESC TEC DE COMERCIO DE CAMPINAS E COLEGIO PRIN...,RUA 203 N 96 SETOR COIMBRA,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN..."
2,2,2016,GOIANIA,133,146,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,3,COLEGIO ESTADUAL MAJOR OSCAR ALVEOLOS,"RUA 2011, UNIDADE 201, S/N, PARQUE ATHENEU","['COLEGIO DINAMICO', 'COLEGIO MARISTA', 'CENTR...","['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ..."
3,3,2016,GOIANIA,146,259,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,1,CAMPUS II - UFG,CAMPUS II - UFG - CONJUNTO ITATIAIA I,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V..."
4,4,2016,GOIANIA,133,244,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,7,ESCOLA PIAGET - PISO SUPERIOR (RUA 1026),"RUA 1.026, QUADRA 68, LOTES 08/25, SETOR PEDRO...","['COLEGIO DINAMICO', 'COLEGIO MARISTA', 'CENTR...","['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ..."


In [63]:
tse['BAIRRO'].isnull().sum()

np.int64(13884)

In [64]:
tse.shape

(305966, 16)

### Ultimas transformações em IBGE para agregação gold

Vamos ter que entender essa tabela como uma amostra, porque eu filtrei teoricamente pela cidade, então, se o censo passar por todas as casas, no minimo deveria ter 1 milhão de linhas esse dataset

In [65]:
# Converter as colunas numéricas para o tipo float
ibge["V005"] = ibge["V005"].str.replace(",", ".").astype(float)
ibge["V009"] = ibge["V009"].str.replace(",", ".").astype(float)
ibge.head(2)

,Unnamed: 0,Nome_do_subdistrito,Nome_do_bairro,Situacao_setor,V001,V002,V005,V007,V009,V011
0,3239,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,276.0,642.0,2432.47,"2543,03",1588.10,"2084,38"
1,3240,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,225.0,491.0,2198.14,"2389,29",1512.69,"2008,52"


In [66]:
# De salarios deixei apenas o que tenho certeza
# Acho que esses dados representam o setor cencitário, mas queremos so valores por setor

ibge = ibge[['Nome_do_subdistrito', 'Situacao_setor', 'V005', 'V009']]

# Converter as colunas numéricas para o tipo float
ibge["V005"] = pd.to_numeric(ibge["V005"], errors="coerce")
ibge["V009"] = pd.to_numeric(ibge["V009"], errors="coerce")


# Agregar os dados por Nome_do_subdistrito
ibge_agrupado = ibge.groupby("Nome_do_subdistrito").agg({
    "Situacao_setor": lambda x: x.mode()[0],  # Pega o valor mais comum
    "V005": "mean",  # Média dos valores
    "V009": "mean",  # Média dos valores
}).reset_index()

# Ver o resultado
ibge_agrupado

,Nome_do_subdistrito,Situacao_setor,V005,V009
0,U.T.P. AEROPORTO,1,3679.232000,2220.962000
1,U.T.P. AEROPORTO INTERNACIONAL SANTA GENOVEVA,1,3435.710000,1542.860000
2,U.T.P. AEROVIARIOS,1,1243.974737,806.283158
3,U.T.P. ALTO DA GLORIA E REDENCAO,1,3067.470400,2045.876400
4,U.T.P. AUTODROMO OU PARQUE LOZANDES,1,8360.048000,4542.501000
...,...,...,...,...
59,U.T.P. VILA PEDROSO,1,880.640800,577.509600
60,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1,1199.879286,750.282143
61,U.T.P. VILA RIZZO,1,874.826000,597.820667
62,U.T.P. ZONA RURAL,1,860.635952,536.335714


In [67]:
ibge_agrupado['Nome_do_subdistrito'].unique()

array(['U.T.P. AEROPORTO',
       'U.T.P. AEROPORTO INTERNACIONAL SANTA GENOVEVA',
       'U.T.P. AEROVIARIOS', 'U.T.P. ALTO DA GLORIA E REDENCAO',
       'U.T.P. AUTODROMO OU PARQUE LOZANDES', 'U.T.P. BAIRRO FELIZ',
       'U.T.P. BALIZA-ITAIPU', 'U.T.P. BUENO', 'U.T.P. CAMPINAS',
       'U.T.P. CAMPUS UNIVERSITARIO/CONJUNTO ITATIAIA',
       'U.T.P. CANDIDA DE MORAIS/MARIA DILCE', 'U.T.P. CAPUAVA',
       'U.T.P. CARAVELAS', 'U.T.P. CEASA/ALDEIA DO VALE',
       'U.T.P. CELINA PARK/RECREIO DOS FUNCIONARIOS', 'U.T.P. CENTRAL',
       'U.T.P. CHACARAS SAO JOAQUIM', 'U.T.P. CIDADE JARDIM',
       'U.T.P. COIMBRA', 'U.T.P. CRIMEIA LESTE', 'U.T.P. FINSOCIAL',
       'U.T.P. GOIANIA II', 'U.T.P. JAO', 'U.T.P. JARDIM AMERICA',
       'U.T.P. JARDIM ATLANTICO',
       'U.T.P. JARDIM BALNEARIO MEIA PONTE/MANSOES GOIANAS',
       'U.T.P. JARDIM EUROPA', 'U.T.P. JARDIM GOIAS',
       'U.T.P. JARDIM GUANABARA', 'U.T.P. JARDIM NOVO MUNDO',
       'U.T.P. JARDIM PETROPOLIS',
       'U.T.P. JARDIM 

#### Agora, vamos queremos juntas ibge com tse

In [68]:
print((tse['BAIRRO'].isnull()).sum())

13884


In [50]:
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable


In [69]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

zonas = pd.read_csv("/home/lisamenezes/pipeline-dados-PDM/data/data/zonas_goiania.csv")

# Função para imputar bairros usando TF-IDF
def imputar_bairros(tse_texts, zonas_texts, zonas_df, threshold=0.7):
    """
    Parâmetros:
    tse_texts: Lista de textos da tabela tse (NM_LOCAL_VOTACAO)
    zonas_texts: Lista de textos da tabela zonas (LOCAL DE VOTACAO)
    zonas_df: DataFrame com dados da tabela zonas
    threshold: Valor mínimo de similaridade para imputação
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit(zonas_texts)

    # Vetorizar os textos
    tse_vetorizado = vectorizer.transform(tse_texts)
    zonas_vetorizado = vectorizer.transform(zonas_texts)

    # Calcular similaridade
    similaridades = cosine_similarity(tse_vetorizado, zonas_vetorizado)

    # Criar lista com bairros imputados
    bairros_imputados = []
    for i, similaridades_linha in enumerate(similaridades):
        max_similaridade = similaridades_linha.max()
        if max_similaridade >= threshold:
            idx = similaridades_linha.argmax()
            bairro = zonas_df.iloc[idx]["BAIRRO"]
        else:
            bairro = None  # Sem similaridade suficiente
        bairros_imputados.append(bairro)

    return bairros_imputados

# Preparar os textos
tse_texts = tse["NM_LOCAL_VOTACAO"].fillna("").tolist()
zonas_texts = zonas["LOCAL DE VOTACAO"].fillna("").tolist()

# Imputar os bairros
bairros_imputados = imputar_bairros(tse_texts, zonas_texts, zonas, threshold=0.7)

# Adicionar a nova coluna 'bairros_imputados' no DataFrame tse
tse["bairros_imputados"] = bairros_imputados

# Exibir o resultado final
tse


,Unnamed: 0,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO,bairros_imputados
0,0,2016,GOIANIA,133,146,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,1,COLEGIO ESTADUAL MAJOR OSCAR ALVEOLOS,"RUA 2011, UNIDADE 201, S/N, PARQUE ATHENEU","['COLEGIO DINAMICO', 'COLEGIO MARISTA', 'CENTR...","['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ...",None
1,1,2016,GOIANIA,2,42,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,3,ESC TEC DE COMERCIO DE CAMPINAS E COLEGIO PRIN...,RUA 203 N 96 SETOR COIMBRA,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",None
2,2,2016,GOIANIA,133,146,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,3,COLEGIO ESTADUAL MAJOR OSCAR ALVEOLOS,"RUA 2011, UNIDADE 201, S/N, PARQUE ATHENEU","['COLEGIO DINAMICO', 'COLEGIO MARISTA', 'CENTR...","['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ...",None
3,3,2016,GOIANIA,146,259,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,1,CAMPUS II - UFG,CAMPUS II - UFG - CONJUNTO ITATIAIA I,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",None
4,4,2016,GOIANIA,133,244,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,7,ESCOLA PIAGET - PISO SUPERIOR (RUA 1026),"RUA 1.026, QUADRA 68, LOTES 08/25, SETOR PEDRO...","['COLEGIO DINAMICO', 'COLEGIO MARISTA', 'CENTR...","['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ...",Setor Pedro Ludovico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305961,305961,2016,GOIANIA,147,311,2,1,VEREADOR,UILLIAN COSTA DA TRINDADE,23500,1,ESCOLA MUNICIPAL ELI BRASILIENSE,RUA DONA CAROLINA ESQ. C/ RUA GV-16 QD 40 N 151,"['COLEGIO DESAFIO', 'COLEGIO ESTADUAL PROFESSO...","['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI...",Residencial Goiânia Viva
305962,305962,2016,GOIANIA,146,169,2,1,VEREADOR,GILMAR BRAZ PEREIRA,18700,1,COLEGIO INTERATIVA (ANTIGO EXITO),RUA MONTE CASTELO NR 108,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",São Judas Tadeu
305963,305963,2016,GOIANIA,146,169,2,1,VEREADOR,UILLIAN COSTA DA TRINDADE,23500,1,COLEGIO INTERATIVA (ANTIGO EXITO),RUA MONTE CASTELO NR 108,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",São Judas Tadeu
305964,305964,2016,GOIANIA,127,229,2,1,VEREADOR,UILLIAN COSTA DA TRINDADE,23500,2,COLEGIO ESTADUAL DE 1O E 2O GRAU ANTONIO OLIVE...,RUA LACY ESQ. C/ RUA ANACA QD 214-A,"['ESCOLA SESI JARDIM PLANALTO', 'COLEGIO ESTAD...","['PRACA ITAPUA QD 30 150', 'RUA C-68 QD 115 33...","['JARDIM PLANALTO', 'VILAS DOS ALPES', 'JARDIM...",None


In [70]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Função para encontrar os melhores matches com um threshold
def match_tfidf(tse_texts, ibge_texts, ibge_df, threshold=0.5):
    """
    Parâmetros:
    tse_texts: Lista de textos da tabela tse
    ibge_texts: Lista de textos da tabela ibge
    ibge_df: DataFrame com dados do IBGE
    threshold: Limite mínimo de similaridade para considerar um match
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit(ibge_texts)

    # Vetorizar os textos
    tse_vetorizado = vectorizer.transform(tse_texts)
    ibge_vetorizado = vectorizer.transform(ibge_texts)

    # Calcular a similaridade de coseno
    similaridades = cosine_similarity(tse_vetorizado, ibge_vetorizado)

    # Encontrar o índice com maior similaridade
    melhores_matches = []
    for i, similaridades_linha in enumerate(similaridades):
        max_similaridade = similaridades_linha.max()
        if max_similaridade >= threshold:
            idx = similaridades_linha.argmax()
            match = ibge_df.iloc[idx].to_dict()
            match['similaridade'] = max_similaridade  # Adicionar a similaridade
        else:
            # Se a similaridade for menor que o threshold, criar linha vazia
            match = {col: None for col in ibge_df.columns}
            match['similaridade'] = max_similaridade
        melhores_matches.append(match)
    
    return melhores_matches

# Criar nova coluna combinada em tse (DS_LOCAL_VOTACAO_ENDERECO + bairros_imputados)
tse["combinado"] = tse["DS_LOCAL_VOTACAO_ENDERECO"].fillna("") + " " + tse["bairros_imputados"].fillna("")

# Criar nova coluna combinada em ibge (Nome_do_subdistrito, ou ajustar conforme necessário)
ibge["combinado"] = ibge["Nome_do_subdistrito"].fillna("")

# Preparar os textos para TF-IDF
tse_texts = tse["combinado"].tolist()
ibge_texts = ibge["combinado"].tolist()

# Definir o threshold mínimo de similaridade
threshold_similaridade = 0.5

# Realizar os matches com threshold
matches = match_tfidf(tse_texts, ibge_texts, ibge, threshold=threshold_similaridade)

# Combinar os DataFrames
tse_matches = pd.DataFrame(matches)
tse_unificado = pd.concat([tse.reset_index(drop=True), tse_matches.reset_index(drop=True)], axis=1)

# Exibir o resultado final
tse_unificado


,Unnamed: 0,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,...,ENDEREÇO,BAIRRO,bairros_imputados,combinado,Nome_do_subdistrito,Situacao_setor,V005,V009,combinado,similaridade
0,0,2016,GOIANIA,133,146,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,...,"['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ...",None,"RUA 2011, UNIDADE 201, S/N, PARQUE ATHENEU",U.T.P. PARQUE AMAZONIA,1.0,1497.56,971.51,U.T.P. PARQUE AMAZONIA,0.568721
1,1,2016,GOIANIA,2,42,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",None,RUA 203 N 96 SETOR COIMBRA,U.T.P. COIMBRA,1.0,1497.84,1264.87,U.T.P. COIMBRA,1.000000
2,2,2016,GOIANIA,133,146,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,...,"['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ...",None,"RUA 2011, UNIDADE 201, S/N, PARQUE ATHENEU",U.T.P. PARQUE AMAZONIA,1.0,1497.56,971.51,U.T.P. PARQUE AMAZONIA,0.568721
3,3,2016,GOIANIA,146,259,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,...,"['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",None,CAMPUS II - UFG - CONJUNTO ITATIAIA I,U.T.P. CAMPUS UNIVERSITARIO/CONJUNTO ITATIAIA,1.0,2322.67,1461.08,U.T.P. CAMPUS UNIVERSITARIO/CONJUNTO ITATIAIA,0.760724
4,4,2016,GOIANIA,133,244,2,1,VEREADOR,ELIAS VAZ DE ANDRADE,40001,...,"['RUA T-37 2693', 'AVENIDA 85, 1440', 'RUA 101...","['SETOR BUENO', 'SETOR MARISTA', 'SETOR PEDRO ...",Setor Pedro Ludovico,"RUA 1.026, QUADRA 68, LOTES 08/25, SETOR PEDRO...",U.T.P. PEDRO LUDOVICO/BELA VISTA/JARDINS DAS E...,1.0,2429.43,1862.88,U.T.P. PEDRO LUDOVICO/BELA VISTA/JARDINS DAS E...,0.545898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305961,305961,2016,GOIANIA,147,311,2,1,VEREADOR,UILLIAN COSTA DA TRINDADE,23500,...,"['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI...",Residencial Goiânia Viva,RUA DONA CAROLINA ESQ. C/ RUA GV-16 QD 40 N 15...,None,NaN,NaN,NaN,None,0.000000
305962,305962,2016,GOIANIA,146,169,2,1,VEREADOR,GILMAR BRAZ PEREIRA,18700,...,"['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",São Judas Tadeu,RUA MONTE CASTELO NR 108 São Judas Tadeu,U.T.P. VILA JARDIM SAO JUDAS TADEU/JARDIM POMPEIA,1.0,1068.57,625.14,U.T.P. VILA JARDIM SAO JUDAS TADEU/JARDIM POMPEIA,0.659521
305963,305963,2016,GOIANIA,146,169,2,1,VEREADOR,UILLIAN COSTA DA TRINDADE,23500,...,"['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",São Judas Tadeu,RUA MONTE CASTELO NR 108 São Judas Tadeu,U.T.P. VILA JARDIM SAO JUDAS TADEU/JARDIM POMPEIA,1.0,1068.57,625.14,U.T.P. VILA JARDIM SAO JUDAS TADEU/JARDIM POMPEIA,0.659521
305964,305964,2016,GOIANIA,127,229,2,1,VEREADOR,UILLIAN COSTA DA TRINDADE,23500,...,"['PRACA ITAPUA QD 30 150', 'RUA C-68 QD 115 33...","['JARDIM PLANALTO', 'VILAS DOS ALPES', 'JARDIM...",None,RUA LACY ESQ. C/ RUA ANACA QD 214-A,None,NaN,NaN,NaN,None,0.000000


In [72]:
tse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305966 entries, 0 to 305965
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Unnamed: 0                 305966 non-null  int64 
 1   ANO_ELEICAO                305966 non-null  int64 
 2   NM_MUNICIPIO               305966 non-null  object
 3   NR_ZONA                    305966 non-null  int64 
 4   NR_SECAO                   305966 non-null  int64 
 5   CD_TIPO_ELEICAO            305966 non-null  int64 
 6   NR_TURNO                   305966 non-null  int64 
 7   DS_CARGO                   305966 non-null  object
 8   NM_VOTAVEL                 305966 non-null  object
 9   NR_VOTAVEL                 305966 non-null  int64 
 10  QT_VOTOS                   305966 non-null  int64 
 11  NM_LOCAL_VOTACAO           305966 non-null  object
 12  DS_LOCAL_VOTACAO_ENDERECO  305966 non-null  object
 13  LOCAL DE VOTACAO           292082 non-null  

In [73]:
# seleção das features boas

tse_unificado = tse_unificado[['ANO_ELEICAO', 'NM_MUNICIPIO', 'NR_ZONA','NR_SECAO', 'CD_TIPO_ELEICAO', 'NR_TURNO', 'DS_CARGO', 'NM_VOTAVEL', 'NR_VOTAVEL', 'QT_VOTOS', 'NM_LOCAL_VOTACAO', 'Nome_do_subdistrito', 'Situacao_setor', 'V005', 'V009']]


In [79]:
tse_unificado.isnull().sum()

ANO_ELEICAO                 0
NM_MUNICIPIO                0
NR_ZONA                     0
NR_SECAO                    0
CD_TIPO_ELEICAO             0
NR_TURNO                    0
DS_CARGO                    0
NM_VOTAVEL                  0
NR_VOTAVEL                  0
QT_VOTOS                    0
NM_LOCAL_VOTACAO            0
Nome_do_subdistrito    113505
Situacao_setor         113505
V005                   113505
V009                   113505
dtype: int64

In [77]:
tse_unificado.to_csv('../data/data/gold/agregacoes.csv')